In [17]:
from pynvml import smi

In [18]:
nvsmi = smi.nvidia_smi.getInstance()

In [19]:
nvsmi

In [20]:
result = nvsmi.DeviceQuery('memory.free, memory.total')

In [21]:
type(result)


dict

In [24]:
nvsmi.DeviceQuery([smi.NVSMI_UTILIZATION_GPU] )

{'gpu': [{'utilization': {'gpu_util': 0, 'unit': '%'}},
  {'utilization': {'gpu_util': 0, 'unit': '%'}}]}